In [1]:
import numpy as np
from qiskit import(QuantumCircuit, execute, Aer)
from qiskit.visualization import plot_histogram
import math
from tkinter import *
import tkinter.messagebox

In [2]:
sim = Aer.get_backend('statevector_simulator')

global circuit
circuit = QuantumCircuit(9, 9)

In [3]:
# #for testing

# move(5, 1, 1, False)
# move(1, 2, 1, True)
# move(5, 2, 2, True)
# circuit.draw()

In [4]:
class MainApp:

    def __init__(self, Parent):
        self.myContainer = Frame(Parent)
        self.winningCondition = False
        self.tempreg1 = 0
        self.tempreg2 = 0
        self.movenum = 1
        self.currmov = 1
        self.status = np.zeros(9)
        
        self.buttons = []
        for i in range(9):
            self.buttons.append(Button(text=" ", wraplength = 150, font='Times 20 bold', bg='gray', 
                                       fg='white', height=4, width=8,command=lambda i=i :self.btnClick(i)))
            self.buttons[i].grid(row = 5 + math.floor(i/3), column = i % 3)
            Grid.rowconfigure(root, i % 3, weight = 1)
            Grid.columnconfigure(root, 5 + math.floor(i/3), weight = 1)
            
        self.ismeasure = StringVar()
        self.choice = Entry(textvariable = self.ismeasure, bd = 5)
        self.choice.grid(row = 1, column = 2)

    def iscurrPlayer(self):
        if self.currmov % 2 == 1:
            return "X"
        else:
            return "O"
    
    def find_res(self, all_results, register, num_qubits):
        for index, element in enumerate(all_results):
            if element != 0:
                ket = bin(index)[2:].zfill(num_qubits+1)
                result = ket[num_qubits - register - 1]
                print(register, ket)
                break
        return result

    #all registers initially 0, if status = 1 --> has qubit, if status = 3 --> has been measured
    def update_status(self, move, register):
        if move != 'm':
            self.status[register] = 1
        else:
            self.status[register] = 3
    
    def disableButton(self):
        for btn in self.buttons:
            btn.configure(state = DISABLED)
    
    def checkForDraw(self):
        draw = 0

        #check if the game is draw after measuring all the boxes
        for element in (self.status):
            if element == 3:
                draw += 1
        
        if draw == 9:
            self.messagebox.showinfo("Qubit Tic-Tac-Toe", "Game: Draw")
            return True
        else:
            return False
    
    def checkForWin(self):
        if(
            self.buttons[0]['text'] == 'O' and self.buttons[1]['text'] == 'O' and self.buttons[2]['text'] == 'O' or
            self.buttons[3]['text'] == 'O' and self.buttons[4]['text'] == 'O' and self.buttons[5]['text'] == 'O' or
            self.buttons[6]['text'] == 'O' and self.buttons[7]['text'] == 'O' and self.buttons[8]['text'] == 'O' or
            self.buttons[0]['text'] == 'O' and self.buttons[3]['text'] == 'O' and self.buttons[6]['text'] == 'O' or
            self.buttons[1]['text'] == 'O' and self.buttons[4]['text'] == 'O' and self.buttons[7]['text'] == 'O' or
            self.buttons[2]['text'] == 'O' and self.buttons[5]['text'] == 'O' and self.buttons[8]['text'] == 'O' or
            self.buttons[0]['text'] == 'O' and self.buttons[4]['text'] == 'O' and self.buttons[8]['text'] == 'O' or
            self.buttons[2]['text'] == 'O' and self.buttons[4]['text'] == 'O' and self.buttons[6]['text'] == 'O' 
        ):
            tkinter.messagebox.showinfo("Qubit Tic-Tac-Toe", "Play O wins")
            return True
        elif(
            self.buttons[0]['text'] == 'X' and self.buttons[1]['text'] == 'X' and self.buttons[2]['text'] == 'X' or
            self.buttons[3]['text'] == 'X' and self.buttons[4]['text'] == 'X' and self.buttons[5]['text'] == 'X' or
            self.buttons[6]['text'] == 'X' and self.buttons[7]['text'] == 'X' and self.buttons[8]['text'] == 'X' or
            self.buttons[0]['text'] == 'X' and self.buttons[3]['text'] == 'X' and self.buttons[6]['text'] == 'X' or
            self.buttons[1]['text'] == 'X' and self.buttons[4]['text'] == 'X' and self.buttons[7]['text'] == 'X' or
            self.buttons[2]['text'] == 'X' and self.buttons[5]['text'] == 'X' and self.buttons[8]['text'] == 'X' or
            self.buttons[0]['text'] == 'X' and self.buttons[4]['text'] == 'X' and self.buttons[8]['text'] == 'X' or
            self.buttons[2]['text'] == 'X' and self.buttons[4]['text'] == 'X' and self.buttons[6]['text'] == 'X' 
        ):
            tkinter.messagebox.showinfo("Qubit Tic-Tac-Toe", "Palyer X wins")
            return True
        else:
            return False

    def checking_moves(self, measure, register1, register2):
        #reg1 = register1 - 1
        #reg2 = register2 - 1
        if measure == True:
            #if we are performing a measure move, we have to check if register has a qubit that is NOT measured
            if self.status[register1] == 1:
                return True                    #can proceed with move() function
            else:
                print("No qubit in slot ", register1, "! Choose again!")
                return False                   #cannot proceed with move() function
        else:
            #move is NOT a measure move so we have to check if registers do not contain a measured qubit
            if self.status[register1] == 3:
                print("Qubit in slot ", register1, "has been measured already! Choose again!")
                return False
            if self.status[register2] == 3:
                print("Qubit in slot ", register2, "has been measured already! Choose again!")
                return False
            return True

    def move(self, register1, register2, player, measure):
        reg1 = register1 
        reg2 = register2
        if measure == True:                               #we measure a register and update the status
            circuit.measure(reg1, reg1)
            self.update_status('m', reg1)
        else:
            if self.status[reg1] == 0:                         #check if registers have qubits, if not, create qubit and update status
                circuit.id(reg1)
                self.update_status('', reg1)
            if self.status[reg2] == 0:
                circuit.id(reg2)
                self.update_status('', reg2)
            if player == 1:                #for player 1, apply Hadamard gate
                circuit.h(reg1)
                circuit.x(reg2)
            else:                          #for player 2, apply X gate
                circuit.h(reg1)
                circuit.x(reg2)

        result = execute(circuit, sim).result().get_statevector()
        return result

    #import pdb
    def btnClick(self, register):
        #pdb.set_trace()
        #for measuring registers
        if self.ismeasure.get() == "m":
            self.tempreg1 = register
            measure_result = self.move(self.tempreg1, self.tempreg1, 0, True)
            usable_result = self.find_res(measure_result, self.tempreg1, 9)
            print("we get:" , usable_result)
            
            if int(usable_result) == 0:
                self.buttons[register]['text'] = "O"
                print("Printed O")
            else:
                self.buttons[register]['text'] = "X"
                print("Printed X")

            if self.checkForWin() or self.checkForDraw(): 
                self.disableButton()    
            return
        
        #if we not measuring, NOTE: Player 1 = X, Player 2 = O
        if self.movenum == 1:
            self.movenum += 1
            self.tempreg1 = register
            self.buttons[register]['text'] = self.iscurrPlayer() + str(math.ceil(self.currmov / 2)) + " " + self.buttons[register]['text']
            print("Player " + self.iscurrPlayer() + " needs to choose another block!")
            return
        else:
            self.movenum = 1
            self.tempreg2 = register
            self.buttons[register]['text'] = self.iscurrPlayer() + str(math.ceil(self.currmov / 2)) + " " + self.buttons[register]['text']
        
        if self.iscurrPlayer() == "X":
            if self.checking_moves(False, self.tempreg1, self.tempreg2):
                print("X is putting qubits down")
                self.move(self.tempreg1, self.tempreg2, 1, False)
        else:
            if self.checking_moves(False, self.tempreg1, self.tempreg2):
                print("O is putting qubits down")
                self.move(self.tempreg1, self.tempreg2, 2, False)
        self.currmov += 1
    
    

In [7]:
#testing if we have an app
root = Tk()
myapp = MainApp(root)
root.mainloop()

Player X needs to choose another block!
X is putting qubits down
Player O needs to choose another block!
O is putting qubits down
Player X needs to choose another block!
X is putting qubits down
0 0000000000
we get: 0
Printed O
1 0000000010
we get: 0
Printed O
4 0000010011
we get: 0
Printed O
